# Plot paired and unpaired synapses

This notebook assumes that all of the studies are orginized into a standard Python data structure. All of the plots are done using plot.ly, so you should look at the appropriate documentation for that if you want to change them.

The variable `study_list` constains a list of studies, where each study is represented as a pyton dictionary. The keys in the dictionary currently include the study name, the subject id, the type of study (aversion, control), and if the fish learned or not. 

The python command `list(study_list[0])` will return a list of the current values that are available for a study.

The study dictionary may also contain a set of calculated synapse pairings.  Each pair is represented as a python dictionary, whose key is the radius with which the pairing was determined, and whose value is a PANDA, with the synapse positions of the pair. Each PANDA has included in it a set of useful values from the study, including the study id, the type of study, the radius for the pair, etc.  Pairs are also transformed using the alignment points to put them into a common coordinate space.  The sets of pairs provided are:

* PairedBefore
* PairedAfter
* UnpairedBefore
* UnpairedAfter
* AlignedPairedBefore
* AlignedPairedAfter
* AlignedUnpairedBefore
* AlignedUnpairedAfter

In [ ]:
# Load up all of the libraries we need

from colour import Color

import synapse_plot_utils as sp
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go

# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

# Load Data

You have two choices, you can reload data from a previous notebook run, or regenerate all of the pairs with data from the catalog. If there have been new studies added, or any data has changed, you must regenerate all the pairs. In general, you should probably only use the save and 

<h3> Restore data from previous run

In [ ]:
dumpfile = 'pairs-dump.pkl'
study_list = sp.restore_studies(dumpfile)

<H3> Load data from catalog

We are going to pull the synapse data down directly from the syapse server.  In order to do this we have to make a connection to the server at synapse.isrd.isi.edu You need to make sure you have logged in using the DERIVA login agent first

In [ ]:
dumpfile = 'pairs-dump.pkl'

all_studies = sp.get_studies()

learners = sp.group_studies(all_studies, group='Type')['learner']
nonlearners = sp.group_studies(all_studies, group='Type')['nonlearner']
aligned = sp.group_studies(all_studies, group='Aligned')[True]

print('{0} learners'.format(len(learners)))
print('{0} nonlearners'.format(len(nonlearners)))
print('{0} aligned'.format(len(aligned)))

# Lets just get the learners and nonlearners for now.
study_list = learners+nonlearners

syn_pair_radii = (4, 300.0,) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold

sp.compute_pairs(study_list, syn_pair_radii)

dumpfile = 'pairs-dump.pkl'

sp.dump_studies(study_list, dumpfile)


<H1> Compute some basic Statistics

In [ ]:
# Set what radius/studies you want to have....

cols = ['ID', 'Type', 'Radius', 'Unpaired Before', 'Unpaired After','Paired Before','Paired After']
c1, c2, c3, c4, c5, c6, c7 = [],[],[],[],[],[],[]

syn_pair_radii = list(study_list[0]['UnpairedBefore'])

for s in study_list:
    for r in syn_pair_radii:
        c1.append(s['Study'])
        c2.append(s['Type'])
        c3.append(r)
        c4.append(len(s['UnpairedBefore'][r].index))
        c5.append(len(s['UnpairedAfter'][r].index))
        c6.append(len(s['PairedBefore'][r].index))
        c7.append(len(s['PairedAfter'][r].index))
    
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6, c7],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'statistics_table')

<H1> Plot histograms of pair distance for all of the studies

In [ ]:
pair_distance = { 'learner': pd.Series(), 'nonlearner' : pd.Series()}

rmax = max(s['PairedAfter'])

for s in study_list:
    # Get the pairing for the maximum radius...
    before = s['PairedBefore'][rmax]
    after = s['PairedAfter'][rmax]
    t = s['Type']
    pair_distance[t] = pair_distance[t].append(pd.Series(np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)))

hist_data_l = go.Histogram(x=pair_distance['learner'], name='Learners Radias {0}'.format(rmax))
hist_data_nl = go.Histogram(x=pair_distance['learner'], name='Nonlearners Radias {0}'.format(rmax))
    
hist_layout = go.Layout(
    title='Pair Distance Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=[hist_data_l, hist_data_nl], layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

<h1> Histogram of pair distances - past a certain pairing threshold 

In [ ]:
thres_pair=20.0 # Threshold for the minimum value to display on the histogram

hist_data_l_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))
hist_data_nl_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))

# Normalized
hist_data_l_N_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")
hist_data_nl_N_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")

hist_layout_Norm = go.Layout(
    title='Pair Distance Histogram Radius Normalized {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig_thres = go.Figure(data=[hist_data_l_thres, hist_data_nl_thres], layout=hist_layout)
hist_fig_Norm_thres = go.Figure(data=[hist_data_l_N_thres, hist_data_nl_N_thres], layout=hist_layout_Norm)

py.offline.iplot(hist_fig_thres, filename='pair-distance_thresholded')
py.offline.iplot(hist_fig_Norm_thres, filename='pair-distance_Norm_thresholded')

<h1> Histogram of intensity ratios

In [ ]:
r  = min(study_list[0]['UnpairedBefore'])

core_ratio = { 'learner': pd.Series(), 'nonlearner' : pd.Series()}
core = { 'learner': pd.Series(), 'nonlearner' : pd.Series()}

for s in study_list:
    before = s['PairedBefore'][r]
    after = s['PairedAfter'][r]
    t = s['Type']
    core_ratio[t] = core_ratio[t].append(pd.Series(before['core'] / after['core']))
    core[t] = core_ratio[t].append(pd.Series(before['core']))
    core[t] = core_ratio[t].append(pd.Series(after['core']))

hist_data_l = go.Histogram(x=core_ratio['learner'], name='Learners Radias {0}'.format(r))
hist_data_nl = go.Histogram(x=core_ratio['learner'], name='Nonlearners Radias {0}'.format(r))
    
hist_layout = go.Layout(
    title='Pair Ratio Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=[hist_data_l, hist_data_nl], layout=hist_layout)

py.offline.iplot(hist_fig, filename='core-intensity')

<H1> Unpaired Centroids

Compute the centroids of the unpaired syapses before/after.  Also find the vector between the before and after centroids.  

In [ ]:
# Select the radius that we want to calculate for

r = min(study_list[0]['UnpairedBefore'])
sl = study_list

# calculate centroids
lbx = [v['UnpairedBefore'][r]['x'].mean() for v in sl if v['Type'] == 'learner']
lby = [v['UnpairedBefore'][r]['y'].mean() for v in sl if v['Type'] == 'learner']
lbz = [v['UnpairedBefore'][r]['z'].mean() for v in sl if v['Type'] == 'learner']

lax = [v['UnpairedAfter'][r]['x'].mean() for v in sl if v['Type'] == 'learner']
lay = [v['UnpairedAfter'][r]['y'].mean() for v in sl if v['Type'] == 'learner']
laz = [v['UnpairedAfter'][r]['z'].mean() for v in sl if v['Type'] == 'learner']

nbx = [v['UnpairedBefore'][r]['x'].mean() for v in sl if v['Type'] == 'nonlearner']
nby = [v['UnpairedBefore'][r]['y'].mean() for v in sl if v['Type'] == 'nonlearner']
nbz = [v['UnpairedBefore'][r]['z'].mean() for v in sl if v['Type'] == 'nonlearner']

nax = [v['UnpairedAfter'][r]['x'].mean() for v in sl if v['Type'] == 'nonlearner']
nay = [v['UnpairedAfter'][r]['y'].mean() for v in sl if v['Type'] == 'nonlearner']
naz = [v['UnpairedAfter'][r]['z'].mean() for v in sl if v['Type'] == 'nonlearner']

# calculate the vector of the centroid differences.
lcx = [ v[0] - v[1] for v in zip(lax, lbx)]
lcy = [ v[0] - v[1] for v in zip(lay, lby)]
lcz = [ v[0] - v[1] for v in zip(laz, lbz)]

ncx = [ v[0] - v[1] for v in zip(nax, nbx)]
ncy = [ v[0] - v[1] for v in zip(nay, nby)]
ncz = [ v[0] - v[1] for v in zip(naz, nbz)]

# Get size of pairings

cnt_lb = [len(v['UnpairedBefore'][r]) for v in sl if v['Type'] == 'learner']
cnt_la = [len(v['UnpairedAfter'][r]) for v in sl if v['Type'] == 'learner']
cnt_nb = [len(v['UnpairedBefore'][r]) for v in sl if v['Type'] == 'nonlearner']
cnt_na = [len(v['UnpairedAfter'][r]) for v in sl if v['Type'] == 'nonlearner']

In [ ]:
ncz

In [ ]:
#Plot the centroids

learner_bcentroids = go.Scatter3d(
        x=lbx, y=lby, z=lbz,
        mode='markers',
        name='Learner Before centroids',
        text = [ 'Study {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(learners,cnt_lb)],
        marker=dict(
            size=5,
            color='rgb(255,0,0)',
            line=dict(
                color='rgb(255,0,0)',
                width=1
            ),
        opacity=0.5
        )
    )

learner_acentroids = go.Scatter3d(
        x=lax, y=lay, z=laz,
        mode='markers',
        name='Learner After centroids',
        text = [ 'Study {0}\\nSize: {1}'.format(v[0], v[1]) for v in zip(learners,cnt_la)],
        marker=dict(
            size=5,
            color='rgb(0,255,0)',
            line=dict(
                color='rgb(0,255,0)',
                width=1
            ),
        opacity=0.5
        )
    )

nonlearner_bcentroids = go.Scatter3d(
        x=nbx, y=nby, z=nbz,
        mode='markers',
        name='Nonlearner Before centroids',
        text = [ 'Study: {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(nonlearners,cnt_nb)],
        marker=dict(
            size=5,
            color='rgb(0,0,255)',
            line=dict(
                color='rgb(0,0,255)',
                width=.5
            ),
        opacity=0.5
        )
    )

nonlearner_acentroids = go.Scatter3d(
        x=nax, y=nay, z=naz,
        mode='markers',
        name='Nonlearner After centroids',
        text = [ 'Study: {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(nonlearners,cnt_na)],
        marker=dict(
            size=5,
            color='rgb(128,128,128)',
            line=dict(
                color='rgb(128,128,128)',
                width=.5
            ),
        opacity=0.5
        )
    )

data = [learner_bcentroids, learner_acentroids, nonlearner_acentroids, nonlearner_bcentroids]

layout = go.Layout(
            height=1000, width=1000,
            title='Centroid Locations',
        )

fig = dict(data=data, layout=layout)
    
py.offline.iplot(fig, filename='unpaired-centroid')



In [ ]:
#Plot the centroids

learner_vcentroids = go.Scatter3d(
        x=lcx, y=lcy, z=lcz,
        mode='markers',
        name='Learner Centroid Vectors',
#        text=learners,
        marker=dict(
            size=5,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

nonlearner_vcentroids = go.Scatter3d(
        x=ncx, y=ncy, z=ncz,
        mode='markers',
        name='Nonlearner Centroid Vectors',
#        text=nonlearners,
        marker=dict(
            size=5,
            line=dict(
            color='rgb(0,0,255)',
            width=0.5
            ),
        opacity=0.5
        )
    )


data = [learner_vcentroids, nonlearner_vcentroids]
layout = go.Layout(
            height=1000, width=1000,
            title='Centroid Vectors',
        )

fig = dict(data=data, layout=layout)
    
py.offline.iplot(fig, filename='centroid-vectors')




<H1> General plotting function for synapses

In [ ]:
def plot_synapses(studylist, 
                  tracelist = ['PairedBefore','PairedAfter','UnpairedBefore','UnpairedAfter'], 
                  slider='Study',
                  radius = False):

    # Use the smallest available radius as the default if one is not provided.
    if not radius:
        radius = min(studylist[0][tracelist[0]])
            
    data = []
    steps = []
        
    # Generate shades of orange and blue for different traces of the same type (in Trace mode)
    if slider == 'Trace':
        shades = 5
    else:
        shades = 1
    lite_orange = Color("orange")
    lite_orange.saturation = .1
    oranges = list(Color('orange').range_to(lite_orange,shades))

    lite_blue = Color("blue")
    lite_blue.saturation = .1
    blues = list(Color('blue').range_to(lite_orange, shades))
    
    black = list(Color('black').range_to(Color('black'), shades))

    # Set Paired to be shades of orange, Unpaired to be shades of blue
    synapsecolors = {
        'PairedBefore' : oranges,
        'UnpairedBefore' : oranges,
        'AlignedPairedBefore' : oranges,
        'AlignedUnpairedBefore' : oranges,
        'PairedAfter' : blues,
        'UnpairedAfter' : blues,
        'AlignedPairedAfter' : blues,
        'AlignedUnpairedAfter' : blues,
        'AlignmentPts' : black
    }
    
    (outer, inner) = (tracelist, studylist) if slider == 'Trace' else (studylist, tracelist)

    maxx,maxy,maxz,minx,miny,minz = -float('inf'),-float('inf'),-float('inf'),float('inf'),float('inf'),float('inf')
    
    for o in outer:
        for i in inner:
            (s, t) = (i, o) if slider == 'Trace' else (o, i)
            synapse_data = s[t][radius]

            maxx = max(maxx,synapse_data['x'].max())
            maxy = max(maxy,synapse_data['y'].max())
            maxz = max(maxz,synapse_data['z'].max())
            minx = min(minx,synapse_data['x'].min())
            miny = min(miny,synapse_data['y'].min())
            minz = min(minz,synapse_data['z'].min())
            
            trace_synapses = go.Scatter3d(
                visible = False,
                x=synapse_data['x'], y=synapse_data['y'], z=synapse_data['z'],
                name='{0} {1}'.format(t, s['Study']),
                mode='markers',
                marker=dict(
                    size=4,
                    # Cycle through the shades.  If we are in Study mode, there is only one....
                    color=synapsecolors[t][len(data) % shades].hex,
                    line=dict(
                        color=synapsecolors[t][len(data) % shades].hex,
                        width=0.5
                ),
                opacity=0.4
                )
            )
            data.append(trace_synapses)
            
        
        if slider == 'Trace':
            steps.append(dict(label='{0}'.format(t)))
        else:
            steps.append(dict(label='{0} ({1})'.format(s['Study'],radius)))
    
    maxx = maxx * 1.04
    maxy = maxy * 1.04
    maxz = maxz * 1.04
    
    mixx = minx * .96
    miny = miny * .96
    mainz = minz * .96

    layout = go.Layout(
                height=900,
                title='Synapses',
                legend=dict(x=.9, xanchor='right'),
                margin=dict(l=100,r=100),
                scene=dict(
                    xaxis=dict(title='X Axis', 
                               range=[minx, maxx], 
                               color='red'),
                    yaxis=dict(title='Y Axis', 
                               range=[miny, maxy], 
                               color = 'green'),
                    zaxis=dict(title='Z Axis', 
                               range=[minz, maxz], 
                               color = 'blue',),
                     
                    aspectmode='cube',
                    dragmode='turntable',
                    
                    annotations = [dict(
                        showarrow = False,
                        x = minx*1.05, y = (maxy-miny)/2+miny, z = (maxz-minz)/2+minz,
                        text = "Medial",
                        font = dict(color = "red",size = 12),
                      ), dict(
                        showarrow = False,
                        x = maxx*.95, y = (maxy-miny)/2+miny, z = (maxz-minz)/2+minz,
                        text = "Lateral",
                        font = dict(color = "red",size = 12),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = miny*1.05, z = (maxz-minz)/2+minz,
                        text = "Posterior",
                        opacity = 0.7,
                        font = dict(color = "green",size = 12),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = maxy*.95, z = (maxz-minz)/2+minz,
                        text = "Anterior",
                        font = dict(color = "green",size = 12),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = (maxy-miny)/2+miny, z = minz*1.05,
                        text = "Dorsal",
                        font = dict(color = "blue",size = 12),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = (maxy-miny)/2+miny, z = maxz*.95,
                        text = "Ventral",
                        font = dict(color = "blue",size = 12),
                      )]
                )
    )
    
    if slider is False:
        for i in data:
            i['visible'] = True
    else: 
        trace_cnt = len(inner)
        # Set first study to be visible
        for i in range(trace_cnt):
            data[i]['visible'] = True
        
        for i,step in enumerate(steps):
            l = ['visible', [False] * len(data)]
            l[1][i*trace_cnt:i*trace_cnt+trace_cnt] = [True] * trace_cnt
            step['args'] = l
            step['method'] = 'restyle'
        
        sliders = [dict(
            active = 0,
            currentvalue = {"visible": False},
            len = .9,
            steps = steps
        )]

        layout['sliders'] = sliders

    fig = dict(data=data, layout=layout)
    py.offline.iplot(fig, filename='synapse-plot')
    

<H1> Plot sets of synapses...

In [ ]:
# Get all radii for a specific study

aligned_studies = [ i for i in sp.group_studies(study_list,group='Aligned')[True] ]
learners = [ i for i in sp.group_studies(study_list,group='Type')['learner'] ]

# All studies...
#s_list = [v[plot_study] for k,v in pairs.items() ]

# Just the learners
#s_list = [ v for k,v in syn_pairs[plot_radius].items() if v['Type'] == 'learner']

# Just the learners that have alignment
s_list = [ s for s in study_list if s['Aligned'] is True and s['Type'] == 'learner']

# Just the studies in the list of names....
s_list = [ s for s in study_list if s['Study'] in ['SynStd6483', 'SynStd7943', 'SynStd7949', 'SynStdR5Y']]

# Pick which traces you would like displayed

#tracelist =['AlignedPairedBefore','AlignedPairedAfter', 'AlignmentPts']
#tracelist = ['PairedBefore','PairedAfter','UnpairedBefore','UnpairedAfter']
tracelist = ['UnpairedBefore','UnpairedAfter']
print('Plotting {0} studies'.format(len(s_list)))

#Pick the slider mode...

slidermode = 'Study'
#slidermode = 'Trace'
#slidermode = False

#plot_synapses(s_list, tracelist, slidermode, 'auto') 
plot_synapses(s_list, tracelist, slidermode) 